# 3. Model Iterations and Plots

By the end of the notebook, you will have run a grid of Schwarzschild models. This will involve the following steps:
1. run a small grid of orbit-based models with DYNAMITE
2. understand the structure of the output
3. plot the output
4. look at the iteration process while running the grid of models

You should run this from the directory ``docs/tutorial_notebooks``.

## Setup

We will run DYNAMITE on CALIFA data of NGC 6278. To prepare the input data files, you should first run the tutorial "Data Preparation for Gauss Hermite kinematics" (``1_data_prep_for_gauss_hermite.ipynb``). The relevant files you need for this tutorial are:

```
| tutorial_notebooks
| ├── NGC6278_input     
| │   ├── dynamite_input                
| │   │   ├── gauss_hermite_kins.ecsv
| │   │   ├── aperture.dat
| │   │   ├── bins.dat 
| │   │   ├── mge.ecsv
| │   │   └── ...
| │   └── ...
| │   └── ...
| └── NGC6278_config.yaml
| └── *.ipynb
|
```

## Read the configuration file 

In [ ]:
import dynamite as dyn

print('DYNAMITE')
print('    version', dyn.__version__)
#print('    installed at ', dyn.__path__)  # Uncomment to print the complete DYNAMITE installation path

fname = 'NGC6278_config.yaml'
c = dyn.config_reader.Configuration(fname, reset_logging=True, reset_existing_output=True)

All the options in the configuration file are held in the object `c`. For example, let's look at the `io_settings`. Output from this tutorial will be saved in the `output_directory`.

In [ ]:
# delete previous output if available
c.remove_existing_orblibs()
c.remove_existing_all_models_file(wipe_other_files=False)
c.backup_config_file(keep=3, delete_other=True)

In [ ]:
c.settings.io_settings

In fact, by creating the configuration object `c`, we have also created the `output_directory` and copied a version of the configuration file there,

In [ ]:
ls NGC6278_output

## Run the models

Making the `ModelIterator` object will start running a grid of orbit-based models. This next step will take about 5-10 minutes using 4 cpus  

In [ ]:
import time

t = time.perf_counter()

smi = dyn.model_iterator.ModelIterator(config=c)

delta_t = time.perf_counter()-t
print(f'Computation time: {delta_t} seconds = {delta_t/60} minutes')

The following files have been created in the models directory,

In [ ]:
ls NGC6278_output/models

Each directory holds a different orbit library 

    orblib_XXX_YYY

where `XXX` labels the iteration when it was created, and `YYY` labels the position within that iteration. Looking inside one of these directories, we see the following files:

In [ ]:
ls NGC6278_output/models/orblib_000_000

which are:

- `cmd_*`: bash scripts for running Fortran programs
- `datfil/`: directory holding the orbit library for the reference potential
- `infil/`: input files for running Fortran programs
- `ml*/`: directories containing output orbital weights (and other results) for different values of `ml`

Each `ml*` directory hold outputs for a re-scaled version of the same potential, where the value of `ml` is a mass scaling applied to a reference potential. The reference potential uses the the first value of `ml` encountered in the parameter search.

Some plots are automatically created,

In [ ]:
ls NGC6278_output/plots

and they represent the following quantities:

1. `kinchi2_progress_plot.png`: chi2 values vs model ID,
2. `kinchi2_plot.png` : model parameters vs chi2 values,
3. `kinematic_map_califa.png`: the kinematic maps for the current minimum-chi2 model.

As an alternative to opening the plot files, we can use the model iterator's method `get_plots()`. It returns these plots as a tuple of objects,

In [ ]:
plots = smi.get_plots()
len(plots)

In [ ]:
# The chi2 vs. model ID plot (kinchi2_progress_plot)
print(type(plots[0]))
plots[0]

In [ ]:
# The model parameters vs chi2 values plot (kinchi2_plot)
# If more than 2 parameters were left free, this would be a traingle plot of chi2 values.
print(type(plots[1]))
plots[1]

The last element of the plots tuple is a list of kinematic maps for the current minimum-chi2 model with one entry for each kinematics dataset. Each list member is itself a tuple of type (`matplotlib.figure.Figure`, `str`) with the string indicating the name of the kinematics.

In [ ]:
# The kinematic maps (kinematic_map_califa)
print(type(plots[2]))
print(plots[2])  # If we had more than one kinematics, the list would be longer
print(plots[2][0])  # The figure and the associated kinematics dataset
print('First (and only) kinematics: ' + plots[2][0][1])  # The name of the kinematics
plots[2][0][0]  # The plot

A summary of all the models run so far is saved in the file `NGC6278_output/all_models.ecsv`. This is an Astropy ECSV file. A table holding this data is stored in `c`, 

In [ ]:
c.all_models.table

At this stage, you could:
    
- run more models, perhaps first adjusting settings in the configuration file, as explained in `2_quickstart.ipynb`,
    - increasing the `n_max_mods` and/or `n_max_iter`
    - adjust parameter bounds and/or which parameters are kept free
- plot other visualisations

## Plotting 

DYNAMITE provides other plotting methods in the `Plotter`:

In [ ]:
plotter = dyn.plotter.Plotter(config=c)

In all the functions that require to use the values of the $\chi^2$ to make the plots, the user can choose which $\chi^2$ to use, by specifying the value of the parameter ``which_chi2``. The recommended value to use is ``which_chi2='kinchi2'``. 

The plots produced by the functions introduced below are all saved in the plot directory specified in the directory ``plots`` within the output directory specified in the configuration file. After running all the cells in this notebook, you will find your plots in the directory ``NGC6278_output/plots``.

The ``mass_plot`` function generates a cumulative mass plot, showing the enclosed mass profiles for the mass-follows-light component (red), for the dark matter (blue), and for the sum of the two (black). The solid lines correspond to the best-fit model, the shaded areas represent 1 sigma uncertainties. You can specify the radial extent of the plot and the type of file you want to be saved with the figure (e.g., ``'.png'``, ``'.pdf'``, ... if ``figtype=None``, the default is used and a ``'.png'`` figure is created).

In [ ]:
fig1 = plotter.mass_plot(which_chi2='kinchi2', Rmax_arcs=50, figtype=None)

The ``orbit_plot`` function generates a plot showing the stellar orbit distribution, described as probability density of orbits; circularity ($\lambda_z$) is represented here as a function of the distance from the galactic centre r (in arcsec). You can specify the type of file you want to be saved with the figure (e.g., ``'.png'``, ``'.pdf'``, ...). In this case, ``Rmax_arcs`` represents the upper radial limit for orbit selection, in arcsec, meaning that only orbits extending up to ``Rmax_arcs`` are plotted.

In [ ]:
fig2 = plotter.orbit_plot(Rmax_arcs=50)

The ``beta_plot`` function generates two plots, showing the intrinsic and projected anisotropy profiles.

In [ ]:
fig3, fig4 = plotter.beta_plot(which_chi2='kinchi2', Rmax_arcs=50)

The ``qpu_plot`` function creates a plot showing the intrinsic flattenings $q$ and $p$, with the blue and black lines respectively, as a function of the distance from the galactic centre (in arcsec). The value of $T = (1-p^2)/(1-q^2)$ is also shown (red line).

In [ ]:
fig5 = plotter.qpu_plot(which_chi2='kinchi2', Rmax_arcs=50,figtype =None)

## The Iteration Process

We will now have a look at the iteration process by examining the $\chi^2$-value.

In the table above (`c.all_models.table`), the model parameters are given for each step of the iteration. Most parameters are fixed and therefore constant, but the mass-to-light ratio `ml` and the dark matter parameter `f-dh` are free. We will examine how these parameters are changed with each iteration.


First, we get the upper and lower limits of the parameters from the configuration file (for more details on this step have a look at the notebook `parameter_space.ipynb`):

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# extract the lo/hi limits for the two free parameters f and ml

f = c.parspace.get_parameter_from_name('f-dh')
f_lims_raw = [f.par_generator_settings['lo'], f.par_generator_settings['hi']]
f_lims = [f.get_par_value_from_raw_value(lim0) for lim0 in f_lims_raw]

ml = c.parspace.get_parameter_from_name('ml')
ml_lims_raw = [ml.par_generator_settings['lo'], ml.par_generator_settings['hi']]
ml_lims = [ml.get_par_value_from_raw_value(lim0) for lim0 in ml_lims_raw]

From the table, we can get a list of all iterations that were made:

In [ ]:
# get list of iterations
iterations = np.unique(c.all_models.table['which_iter'])

Now plot the $\chi^2$-value of the models for every iteration:

In [ ]:
# plot chi2 of models vs iterations
for iter0 in iterations:
    table = c.all_models.table
    table = table[table['which_iter']==iter0]
    plt.figure(figsize=(6,5))
    plt.scatter(table['f-dh'],
                table['ml'],
                c=table['chi2'],
                cmap=plt.cm.viridis_r,
                s=200)


    cbar = plt.colorbar()
    cbar.set_label(r'$\chi^2$')
    plt.gca().set_title(f'iteration {iter0}')

    plt.gca().set_xlim(*f_lims)
    plt.gca().set_ylim(*ml_lims)
    plt.gca().set_xlabel(f.LaTeX)
    plt.gca().set_ylabel(ml.LaTeX)
    plt.gca().set_xscale('log')
    plt.tight_layout()
    plt.show()

We can see how the parameter values get closer together for each iteration. The next plot gives an overview over all iterations:

In [ ]:
# plot the models: f-dh vs ml altogether
plt.scatter(c.all_models.table['f-dh'],
            c.all_models.table['ml'],
            c=c.all_models.table['chi2'] - np.min(c.all_models.table['chi2']),
            cmap=plt.cm.viridis_r,
            s=200)
cbar = plt.colorbar()
cbar.set_label(r'$\chi^2$')
plt.gca().set_title(f'all iterations')
plt.gca().set_xlim(*f_lims)
plt.gca().set_ylim(*ml_lims)
plt.gca().set_xlabel(f.LaTeX)
plt.gca().set_ylabel(ml.LaTeX)
plt.gca().set_xscale('log')
plt.tight_layout()
plt.show()

We can see how with every iteration, the $\chi^2$-value becomes smaller, as does the scatter between the values.

The plot `kinchi2_plot` already shown above is another representation of the iteration process. The different models are plotted on a grid in the parameter space, where the best fit model is marked with a cross. Above we stored this plot in `plots[1]`,

In [ ]:
plots[1]